# Sesion 2 del proyecto de LNR

## Estudiantes
- Ignacio Cano Navarro
- Angel Langdon Villamayor

## Primeros modelos



### Preproceso

Importaremos las librerías y el preprocesado realizado en la entrega anterior

In [ ]:
import os
import re

import pandas as pd
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


# necessary packages
nltk.download("stopwords")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# Preprocessing
def delete_stop_words(comment):
    spanish_stopwords = stopwords.words("spanish")
    return " ".join([w for w in comment.split() if w not in spanish_stopwords])

def steam(text, stemmer):
    stemmed_text = [stemmer.stem(word) for word in word_tokenize(text)]
    return " ".join(stemmed_text)

def clean_text_column(df, col, stemmer):
    """Normalizes a string column to have a processed format 
    Arguments:
      df (pd.DataFrame): the dataframe that contains the column to normalize
      col (str): the dataframe column to normalize
      steammer (nltk.steam.SnowballStammer): the steammer to use for 
          steamming the text
    Returns:
      The dataframe with the preprocessed column
    """
    df = df.copy() # copy the dataframe avoid modifying the original
    # Make the comments to lowercase 
    df[col] = df[col].str.lower()
    # Delete the stop words
    df[col] = [delete_stop_words(c) for c in df[col]]
    # Replace underscores and hyphens with spaces 
    df[col] = df[col].str.replace("_", " ")
    df[col] = df[col].str.replace("-", " ")
    # Create the regex to delete the urls, usernames and emojis
    urls = r'https?://[\S]+'
    users = r'@[\S]+'
    emojis = r'[\U00010000-\U0010ffff]'
    hashtags = r'\s#[\S]+'
    # Join the regex
    expr = f'''({"|".join([urls,
                           users,
                           hashtags,
                           emojis])})'''
    # Replace the urls, users and emojis with empty string
    df[col] = df[col].str.replace(expr, "", regex=True)                      
    # Get only the words of the text
    df[col] = df[col].str.findall("\w+").str.join(" ")
    # Delete the numbers
    df[col] = df[col].str.replace("[0-9]+", "",regex=True)
    # Steam the words of the text for each text in the specified column
    #df[col] = [steam(c, stemmer) for c in  df[col]]
    return df


# Initialize the steammer to Spanish language
stemmer = SnowballStemmer('spanish')
# read the data
df_original = pd.read_csv("train.csv") 
# Normalize the "comment" column
df = clean_text_column(df_original, "comment", stemmer)
df.head()


,topic,thread_id,comment_id,reply_to,comment_level,comment,argumentation,constructiveness,positive_stance,negative_stance,target_person,target_group,stereotype,sarcasm,mockery,insult,improper_language,aggressiveness,intolerance,toxicity,toxicity_level
0,CR,0_000,0_002,0_002,1,pensó zumo restar,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1
1,CR,0_001,0_003,0_003,1,gusta afeitado seco gente,0,0,0,0,0,1,1,1,1,0,0,0,0,1,1
2,CR,0_002,0_004,0_004,1,asi gusta maten alta mar mas inmigrantes asi p...,0,0,0,0,0,1,0,0,0,0,0,1,1,1,2
3,CR,0_003,0_005,0_005,1,loss mas valientes mejor cortan cabezas vosotr...,0,0,0,0,1,1,0,1,1,0,0,0,0,1,1
4,CR,0_004,0_006,0_006,1,costumbres,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1


In [ ]:
# Create a TF-IDF with ngrams 
tfidf = TfidfVectorizer(ngram_range=(1,1))
# Fit with the comments 
features = tfidf.fit_transform(df["comment"])
# Get the feature extraction matrix
df_features = pd.DataFrame(features.todense(),
             columns= tfidf.get_feature_names())
# Print the first comment
print(df_original["comment"].iloc[0])
# Print the sorted by probability first row of the matrix
df_features.sort_values(by=0, axis=1, ascending=False).head(1)

Pensó: Zumo para restar.


,pensó,restar,zumo,opino,operativas,opinadores,opinamos,opinan,opinando,opinar,opine,opines,opinion,opiniones,opinión,opniones,operación,oponen,oportunas,oportunidad,oportunidades,oposicion,oposición,opresor,opresores,oprimidas,oprimido,oprimidos,operandi,operaciones,optas,onegetas,olía,olímpicamente,omiso,omite,omiten,omites,omitir,omnipresente,...,elijo,emigraban,emigrado,emigramos,emigran,emigrante,emigrantes,emigrar,emigraran,emigraron,emigré,emigró,embarcación,embarcaciones,embajadss,embajadas,elimina,eliminación,eliminar,eliminas,elimine,elisa,elite,eljueves,ella,ellas,elle,ello,ellos,elmundotoday,elpais,elplural,elsaltodiario,elíptica,em,ema,email,emanan,emanuel,útiles
0,0.630681,0.608146,0.482079,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Primer modelo

## Consideraciones

Se va a utilizar gridsearchCV de sklearn para encontrar los parámetros óptimos de cada uno de los modelos a utilizar. 

También se va a utilizar el parámetro random_state para que los resultados sean los mismos siempre.

Aunque quede fuera del alcanze de este informe se va a intentar una sencilla implementación en Keras.

## Variable Toxicity

Vamos a empezar con la predicción de la variable toxicity. (binaria) Posteriormente pasaremos a predecir la variable toxicity_level, la cual suponemos que será mucho más complicada para los siguientes modelos que implementaremos de predecir. 

## SVM

Estos métodos están propiamente relacionados con problemas de clasificación. Dado un conjunto de ejemplos de entrenamiento (de muestras) podemos etiquetar las clases y entrenar una SVM para construir un modelo que prediga la clase de una nueva muestra. Intuitivamente, una SVM es un modelo que representa a los puntos de muestra en el espacio, separando las clases a 2 espacios lo más amplios posibles mediante un hiperplano de separación definido como el vector entre los 2 puntos, de las 2 clases, más cercanos al que se llama vector soporte. Cuando las nuevas muestras se ponen en correspondencia con dicho modelo, en función de los espacios a los que pertenezcan, pueden ser clasificadas a una o la otra clase.



Vamos a empezar pues cargando las librerías necesarias para implementar los diferentes modelos.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, make_scorer
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

f1 = make_scorer(f1_score , average='macro')


Se ha utilizado la función GridSearchCV de sklearn con el objetivo de probar diferentes parámetros y encontrar el óptimo. 

In [ ]:
parameters = {'kernel':['poly', 'rbf', 'sigmoid'], 'degree':[1, 3, 5], 'C':[0.5, 1, 1.5]}

X,y = df_features, df_original['toxicity']
X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=0.2, random_state=20)

clf = GridSearchCV(svm.SVC(class_weight='balanced', random_state=20), parameters, scoring = f1)
clf.fit(X_train, y_train)
pred = clf.predict(X_eval)
score = f1_score(y_eval, pred, average='macro')
print(score)
print(clf.get_params())
clf.best_params_

0.59490103021183
{'cv': None, 'error_score': nan, 'estimator__C': 1.0, 'estimator__break_ties': False, 'estimator__cache_size': 200, 'estimator__class_weight': 'balanced', 'estimator__coef0': 0.0, 'estimator__decision_function_shape': 'ovr', 'estimator__degree': 3, 'estimator__gamma': 'scale', 'estimator__kernel': 'rbf', 'estimator__max_iter': -1, 'estimator__probability': False, 'estimator__random_state': 20, 'estimator__shrinking': True, 'estimator__tol': 0.001, 'estimator__verbose': False, 'estimator': SVC(C=1.0, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=20, shrinking=True, tol=0.001,
    verbose=False), 'iid': 'deprecated', 'n_jobs': None, 'param_grid': {'degree': [3]}, 'pre_dispatch': '2*n_jobs', 'refit': True, 'return_train_score': False, 'scoring': make_scorer(f1_score, average=macro), 'verbose': 0}


{'degree': 3}

## Decision Trees

Un árbol de decisión es un modelo predictivo que divide el espacio de los predictores agrupando observaciones con valores similares para la variable respuesta o dependiente.

Para dividir el espacio muestral en sub-regiones es preciso aplicar una serie de reglas o decisiones, para que cada sub-región contenga la mayor proporción posible de individuos de una de las poblaciones.



In [ ]:
tree_para = {'criterion':['gini', 'entropy']}

clf = GridSearchCV(DecisionTreeClassifier(class_weight='balanced', random_state=20), tree_para, scoring=f1)

clf.fit(X_train, y_train)
pred = clf.predict(X_eval)
score = f1_score(y_eval, pred, average='macro')

print(score)
print(clf.get_params())

## Random Forest

Los árboles de decisión tienen la tendencia de sobre-ajustar (overfit). Esto quiere decir que tienden a aprender muy bien los datos de entrenamiento pero su generalización no es tan buena. Una forma de mejorar la generalización de los árboles de decisión es usar regularización. Para mejorar mucho más la capacidad de generalización de los árboles de decisión, deberemos combinar varios árboles

Un Random Forest es un conjunto (ensemble) de árboles de decisión combinados con bagging. Al usar bagging, lo que en realidad está pasando, es que distintos árboles ven distintas porciones de los datos. Ningún árbol ve todos los datos de entrenamiento. Esto hace que cada árbol se entrene con distintas muestras de datos para un mismo problema. De esta forma, al combinar sus resultados, unos errores se compensan con otros y tenemos una predicción que generaliza mejor.

Es por eso que vamos a probar el método de Random Forest a ver si este mejora el resultado de Decision Trees

In [ ]:
rf = RandomForestClassifier(n_jobs=-1, oob_score = True, class_weight='balanced', random_state=20) 

param_grid = { 
    'n_estimators': [100, 150, 200],
    'max_features': ['auto', 'log2']
}

clf = GridSearchCV(estimator=rf, param_grid=param_grid, scoring=f1)
clf.fit(X_train, y_train)

pred = clf.predict(X_eval)
score = f1_score(y_eval, pred, average='macro')
print(score)
print(print(clf.get_params()))

0.6144942900663756
{'cv': None, 'error_score': nan, 'estimator__bootstrap': True, 'estimator__ccp_alpha': 0.0, 'estimator__class_weight': 'balanced', 'estimator__criterion': 'gini', 'estimator__max_depth': None, 'estimator__max_features': 'auto', 'estimator__max_leaf_nodes': None, 'estimator__max_samples': None, 'estimator__min_impurity_decrease': 0.0, 'estimator__min_impurity_split': None, 'estimator__min_samples_leaf': 1, 'estimator__min_samples_split': 2, 'estimator__min_weight_fraction_leaf': 0.0, 'estimator__n_estimators': 100, 'estimator__n_jobs': -1, 'estimator__oob_score': True, 'estimator__random_state': 20, 'estimator__verbose': 0, 'estimator__warm_start': False, 'estimator': RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
           

## Logistic Regression

La Regresión Logística, es un método de regresión que permite estimar la probabilidad de una variable cualitativa binaria en función de una o más variables cuantitativas. Una de las principales aplicaciones de la regresión logística es la de clasificación binaria, en el que las observaciones se clasifican en un grupo u otro dependiendo del valor que tome la variable empleada como predictor.

In [ ]:
param_grid = { 
    'C': [0.5, 1, 2],
    'solver':['liblinear', 'lbfgs', 'newton-cg']
}

clf = GridSearchCV(LogisticRegression(class_weight='balanced', random_state=20), param_grid, scoring = f1)
clf.fit(X_train, y_train)
pred = clf.predict(X_eval)
score = f1_score(y_eval, pred, average='macro')
print(score)
print(clf.get_params())

0.6503144499280026
{'cv': None, 'error_score': nan, 'estimator__C': 1.0, 'estimator__class_weight': 'balanced', 'estimator__dual': False, 'estimator__fit_intercept': True, 'estimator__intercept_scaling': 1, 'estimator__l1_ratio': None, 'estimator__max_iter': 100, 'estimator__multi_class': 'auto', 'estimator__n_jobs': None, 'estimator__penalty': 'l2', 'estimator__random_state': 20, 'estimator__solver': 'lbfgs', 'estimator__tol': 0.0001, 'estimator__verbose': 0, 'estimator__warm_start': False, 'estimator': LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=20, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False), 'iid': 'deprecated', 'n_jobs': None, 'param_grid': {'C': [0.5, 1, 2], 'solver': ['liblinear', 'lbfgs', 'newton-cg']}, 'pre_dispatch': '2*n_jobs', 'refit': True

Como podemos ver, en general los modelos utilizados no son capaces de superar el 0.66 de F1-score en la predicción de la variable toxicity. Esto puede ser debido a la variabilidad del dataset, unido a que son modelos sencillos y que con los pocos datos que disponemos, no son capaces de generalizar lo suficiente.

## Variable Toxicity_Level

Ahora procederemos a aplicar los mismos modelos a la variable Toxicity_level

## SVM

In [ ]:
parameters = {'degree':[1,3,5],
              'kernel': ['poly', 'rbf'],
              'C':[0.5,1,1.5]}

X,y = df_features, df_original['toxicity_level']
X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=0.2, random_state=20)

clf = GridSearchCV(svm.SVC(class_weight='balanced', random_state=20), parameters, scoring = f1)
clf.fit(X_train, y_train)
pred = clf.predict(X_eval)
score = f1_score(y_eval, pred, average='macro')
print(score)
print(clf.get_params())

KeyboardInterrupt: ignored

## Decision Trees

In [ ]:
tree_para = {'criterion':['gini', 'entropy']}

clf = GridSearchCV(DecisionTreeClassifier(DecisionTreeClassifier(class_weight='balanced', random_state=20)), tree_para, scoring=f1)

clf.fit(X_train, y_train)
pred = clf.predict(X_eval)
score = f1_score(y_eval, pred, average='macro')

print(score)
print(clf.get_params())

## Random Forest

In [ ]:
rf = RandomForestClassifier(n_jobs=-1, oob_score = True, class_weight='balanced', random_state=20) 

param_grid = { 
    'n_estimators': [100, 150, 200],
    'max_features': ['auto', 'log2']
}

clf = GridSearchCV(estimator=rf, param_grid=param_grid, scoring=f1)
clf.fit(X_train, y_train)

pred = clf.predict(X_eval)
score = f1_score(y_eval, pred, average='macro')
print(score)
print(print(clf.get_params()))

## Logistic Regression

In [ ]:
param_grid = { 
    'solver':['liblinear', 'lbfgs', 'newton-cg']
}

clf = GridSearchCV(LogisticRegression(class_weight='balanced', random_state=20), param_grid, scoring = f1)
clf.fit(X_train, y_train)
pred = clf.predict(X_eval)
score = f1_score(y_eval, pred, average='macro')
print(score)
print(clf.get_params())

Como hemos previsto, el F1-score ha sido mucho menor al intentar predecir la variable toxicity_level, de hecho, el mejor modelo en este caso ha sido logistic regression con un 0.35 de F1-score. Para posteriores informes seria interesante aplicar CV para unos resultados mas exactos

# BONUS

Implementamos un modelo secuencial de keras con una capa de convolución.
Como no es el objetivo de este informe, sino que se hace como algo "extra" no se explicará con tanto detalle ya que simplemente queríamos comprobar como funcionaba esta librería para el procesamiento de textos y el F1-score que devolvía dicho modelo. 

## Preprocesamiento


In [ ]:
text_data = list(df['comment'])
leng = [len(txt) for txt in text_data]
X,y = df['comment'], df['toxicity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=20)
X_train, X_test, y_train, y_test = list(X_train), list(X_test), np.array(y_train), np.array(y_test)


## Tranformamos el input con el word ebmedding de keras

In [ ]:
from keras.preprocessing.text import Tokenizer



tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_data)

## Codificamos train y test

In [ ]:
train_data = tokenizer.texts_to_sequences(X_train)
test_data = tokenizer.texts_to_sequences(X_test)

## Rellenamos los "huecos" con 0 para que todas las cadenas tengan la misma longitud 

In [ ]:
from keras.preprocessing.sequence import pad_sequences

train_data = pad_sequences(train_data, maxlen=110, padding="post") 
test_data = pad_sequences(test_data, maxlen=110, padding="post") 
vocab_size = len(tokenizer.word_index) + 1


## Creamos pesos para la clase minoritaria

In [ ]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train),
                                                 y_train)
class_weights = {k:v for k,v in enumerate(class_weights)}
class_weights

In [ ]:
import keras.backend as K

def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

## Creación de la topología con una capa de convolución

In [ ]:
from keras.models import Sequential
from keras import layers

embedding_dim = 100

model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, input_length=110, trainable=True))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', get_f1])
model.summary()


## Entrenamos el modelo

In [ ]:
history = model.fit(train_data, y_train,
                    epochs=15,
                    verbose=False,
                    validation_data=(test_data, y_test),
                    batch_size=10,
                    class_weight=class_weights)
loss, accuracy, f1 = model.evaluate(train_data, y_train, verbose=False)
print(f"Training Accuracy: {accuracy}, Training F1: {f1}")
loss, accuracy, f1 = model.evaluate(test_data, y_test, verbose=False)
print(f"Test Accuracy: {accuracy}, Test F1: {f1}")


## Resultado

Podemos ver como el modelo se sobreajusta en los datos de train, pero sin embargo no es capaz de generalizar lo suficiente como para conseguir un buen f1-score en los datos de test. Para ello sería necesario aumentar el dataset. 